# 数据挖掘第六章作业 第15题

# (a) 挖掘合著者关系 

1. 频繁模式挖掘也适用于寻找论文的合著者关系。
1. 可以把一篇论文的合著者看成是一条事务，在事务中，每个合著者是一个项。
1. 用Apriori算法或其他频繁项集挖掘的实现，挖掘合著者关系的频繁项集。
1. 处在频繁项集中的合著者，可以认为是经常进行合作的合著者关系。

我们可以复用7题的代码
先在小型的合著者关系数据集上测试一下代码

In [21]:
# 读取数据
import pandas as pd
def load_data(path):
        data = pd.read_csv(path)
        num = data.iloc[:, -2]
        y = data.iloc[:, -1]

        return num, y

# 生成事务合集，每篇文章作为一个集合
def getShoppingCart(orders, items):
    numOfOrder = len(set(orders))
    shoppingCart = [set() for _ in range(numOfOrder)]
    for j in range(len(orders)):
        shoppingCart[orders[j]-1].add(items[j])
    return shoppingCart

orders, items = load_data('mini.csv')
shoppingCart = getShoppingCart(orders, items)
print(shoppingCart)

[{'Jiawei Han', 'Yan Shi'}, {'Xiaofeng Meng', 'Jian Pei', 'Yan Shi'}, {'Laks V. S. Lakshmanan', 'Xiaokui Xiao', 'Jian Pei'}, {'Yufei Tao', 'Xiaokui Xiao', 'Wang-Chien Lee'}, {'Zhenhui Li', 'Wang-Chien Lee', 'Jiawei Han'}, {'Jiawei Han', 'Hong Cheng', 'Xifeng Yan'}, {'Jiawei Han', 'Xifeng Yan', 'Philip S. Yu'}, {'Jiawei Han', 'Hong Cheng', 'Dong Xin'}, {'Hong Cheng', 'Xifeng Yan', 'Dong Xin'}, {'Hong Cheng', 'Xifeng Yan', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}, {'Yuqing Wu', 'Kun-Lung Wu', 'Philip S. Yu'}]


In [22]:
FREQUENT = 5


class Apriori:

    # 判断itemList是否在order中
    def __contains(self, order, itemList):
        for item in itemList:
            if item not in order:
                return False
        return True

    # 计算itemList在shoppingCart中出现的次数
    def count(self, itemList, shoppingCart):
        count = 0
        for order in shoppingCart:
            if (self.__contains(order, itemList)):
                count += 1
        return count

    # 生成下一层的itemList
    def __generate_next_itemList(self, itemList):
        next_itemList = []
        for i in range(len(itemList)):
            for j in range(i+1, len(itemList)):
                if itemList[i][:-1] == itemList[j][:-1]:
                    newList = [x for x in itemList[i]]
                    newList.append(itemList[j][-1])
                    next_itemList.append(newList)
        return next_itemList

    # 生成第一层的itemList
    def __getFirstItemLists(self, items):
        firstItemList = [[x] for x in set(items)]
        return firstItemList

    # 过滤掉不满足最小支持度的itemList
    def __filter(self, itemLists, shoppingCart):
        filterList = []
        for itemList in itemLists:
            if self.count(itemList, shoppingCart) >= FREQUENT:
                filterList.append(itemList)
        return filterList

    # 主函数,也是对外接口
    def analyze(self, items, shoppingCart):
        itemLists = self.__getFirstItemLists(items)
        totalFrequentItemLists = []

        while len(itemLists) > 0:
            filteredItemLists = []
            filteredItemLists = self.__filter(itemLists, shoppingCart)
            if (len(filteredItemLists) > 0):
                totalFrequentItemLists.append(filteredItemLists)
            nextItemList = self.__generate_next_itemList(filteredItemLists)
            itemLists = nextItemList

        return totalFrequentItemLists


In [23]:
# 输出所有的频繁项集
ap = Apriori()
print(ap.analyze(items, shoppingCart))

[[['Jiawei Han'], ['Kun-Lung Wu'], ['Philip S. Yu'], ['Yuqing Wu']], [['Kun-Lung Wu', 'Philip S. Yu'], ['Kun-Lung Wu', 'Yuqing Wu'], ['Philip S. Yu', 'Yuqing Wu']], [['Kun-Lung Wu', 'Philip S. Yu', 'Yuqing Wu']]]


# (b) 频繁模式挖掘的评估度量

本章讨论的评估度量包括：
- 提升度
- 卡方值
- 全置信度
- 最大置信度
- Kulc值
- 余弦度量

在论文数据集这样的大型数据集中，作者的出现往往是非常稀疏的。因此，存在着大量的零事务，所有度量的零不变性是很重要的。

具有零不变性的度量有：全置信度, 最大置信度, Kulc值, 余弦度量

In [24]:
def conditionP(a, b):
    countb = ap.count([b], shoppingCart)
    coountab = ap.count([a, b], shoppingCart)
    return coountab/countb



ans = ap.analyze(items, shoppingCart)
two = ans[1]
for pair in two:
    a = pair[0]
    b = pair[1]
    max_confidence = max(conditionP(a, b), conditionP(b, a))
    all_confidence = min(conditionP(a, b), conditionP(b, a))
    Kulc = 0.5*(conditionP(a, b) + conditionP(b, a))
    cosine = (conditionP(a, b) * conditionP(b, a))**0.5
    print(pair)
    print('max_confidence:', max_confidence)
    print('all_confidence:', all_confidence)
    print('Kulc:', Kulc)
    print('cosine:', cosine)
    print('----------------')




['Kun-Lung Wu', 'Philip S. Yu']
max_confidence: 1.0
all_confidence: 0.8181818181818182
Kulc: 0.9090909090909092
cosine: 0.9045340337332909
----------------
['Kun-Lung Wu', 'Yuqing Wu']
max_confidence: 1.0
all_confidence: 1.0
Kulc: 1.0
cosine: 1.0
----------------
['Philip S. Yu', 'Yuqing Wu']
max_confidence: 1.0
all_confidence: 0.8181818181818182
Kulc: 0.9090909090909092
cosine: 0.9045340337332909
----------------


Kulc 和 余弦度量考虑了合著者双方的条件概率，而且不受零事务的影响，应当是布景会的指标。

# (c) 导师关系和指导周期

考虑学生和导师的关系。

当学生发表论文时，一般作者中会包含导师的名字。

但是当导师发表论文时，一般作者中不一定包含学生的名字。

因此，导师和学生双方的关系是不对称的，他们的条件概率也是不对称的，应该相差较大。

所以，我们可以考虑计算频繁模式中的全置信度和最大置信度，通过它们的差，来判断导师和学生的关系。

In [25]:
for pair in two:
    a = pair[0]
    b = pair[1]
    max_confidence = max(conditionP(a, b), conditionP(b, a))
    all_confidence = min(conditionP(a, b), conditionP(b, a))
    print(pair)
    print('difference: ', max_confidence - all_confidence)
    print('----------------')
    

['Kun-Lung Wu', 'Philip S. Yu']
difference:  0.18181818181818177
----------------
['Kun-Lung Wu', 'Yuqing Wu']
difference:  0.0
----------------
['Philip S. Yu', 'Yuqing Wu']
difference:  0.18181818181818177
----------------


### 在某个频繁模式中，最大置信度和全置信度的差较大时，可以猜测两者有导师学生关系。

关于指导的近似周期，可以分时段统计频繁模式，然后计算频繁模式中的全置信度和最大置信度的差，通过差的变化趋势，来判断指导周期。

当指导周期结束后，两人的关系会发生变化，或者两人不再合作，或者两人的关系会变得更加对称，这都会使得全置信度和最大置信度的差变小。于是我们可以想定差较大的部分就是两人的指导周期。